In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import gzip

/opt/anaconda3/envs/mindscope_utilities/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
files = os.listdir()
pklidx = np.array(['.pkl' in f for f in files])
mousefiles = np.array(files)[pklidx]

for fn in tqdm(mousefiles[1:]):
    df = pd.read_pickle(fn)

    # Create smaller df with only essential columns
    unneeded = ['exposure_level','sex','cre_line','targeted_structure','imaging_depth','ophys_experiment_id','dff']
    df.index = df.index.astype('int16')
    df_sm = df.drop(unneeded,axis=1)

    # Explode out the trace arrays
    df = df.explode(column='trace').reset_index(drop=True)
    # Add the corresponding timestamps
    df['time'] = np.tile(np.linspace(-0.5,0.7333333333333334,38),int(len(df)/38))
    # Pivot to multi-indexed df
    lmao = df.pivot(index=['ophys_session_id','stimulus_presentations_id','time'],columns='cell_specimen_id',values='trace')
    # Hack to create 3D matrix by stacking along the time dimension
    time_data = []
    for i, ti in enumerate(np.linspace(-0.5,0.7333333333333334,38)):
        val = np.half(lmao.loc[:,:,ti].values)
        time_data.append(val)
    time_data = np.stack(time_data,axis=2)
    
    # Save
    f = gzip.GzipFile(fn[:-4]+'_dffts.npy.gz', "w")
    np.save(f, time_data)
    f.close()

    # df.pivot doesn't work if you have previously cast values to float16!
    df_sm = df_sm.astype({'dff_bc':'float64', 'dff_stim500':'float64'})
    
    dff_bc = np.half(df_sm.pivot(index=['ophys_session_id','stimulus_presentations_id'],columns='cell_specimen_id',values='dff_bc').values)
    f = gzip.GzipFile(fn[:-4]+'_dffbc.npy.gz', "w")
    np.save(f, dff_bc)
    f.close()

    dff_stim500 = np.half(df_sm.pivot(index=['ophys_session_id','stimulus_presentations_id'],columns='cell_specimen_id',values='dff_stim500').values)
    f = gzip.GzipFile(fn[:-4]+'_dffstim.npy.gz', "w")
    np.save(f, dff_stim500)
    f.close()

    img = df_sm.pivot(index=['ophys_session_id','stimulus_presentations_id'],columns='cell_specimen_id',values='image_name').values[:,0]
    chg = df_sm.pivot(index=['ophys_session_id','stimulus_presentations_id'],columns='cell_specimen_id',values='is_change').values[:,0]
    omi = df_sm.pivot(index=['ophys_session_id','stimulus_presentations_id'],columns='cell_specimen_id',values='omitted').values[:,0]
    ses = df_sm.pivot(index=['ophys_session_id','stimulus_presentations_id'],columns='cell_specimen_id',values='session_number').values[:,0]
    label_data = np.stack([img,chg,omi,ses], axis=1)
    f = gzip.GzipFile(fn[:-4]+'_label.npy.gz', "w")
    np.save(f, label_data)
    f.close()
    

100%|██████████| 39/39 [16:12<00:00, 24.92s/it]
